#Notebook for AnnData exploration, processing, and stacking. 

In [1]:
import jax
import jaxlib

print("jax version:", jax.__version__)
print("jaxlib version:", jaxlib.__version__)

import scvi
import h5py
import anndata as ad
import pandas as pd
import scipy.sparse as sp
import numpy as np


jax version: 0.5.0
jaxlib version: 0.5.0


/gpfs/commons/home/svaidyanathan/miniconda3/envs/scvi-env/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/gpfs/commons/home/svaidyanathan/miniconda3/envs/scvi-env/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/gpfs/commons/home/svaidyanathan/miniconda3/envs/scvi-env/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/gpfs/commons/home/svaidyanathan/miniconda3/envs/scvi-env/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing CSCDataset from `anndata.experimental` is deprecated. Import anndata.abc.CSCDataset instead.
  warnings.warn(msg, FutureWarning

Reading in the ATSE annData with alternative splicing input data. 

In [2]:
atse_anndata = ad.read_h5ad('/gpfs/commons/groups/knowles_lab/Karin/TMS_MODELING/DATA_FILES/BRAIN_ONLY/02072025/TMS_Anndata_ATSE_counts_with_waypoints_20250207_153520.h5ad')
atse_anndata

AnnData object with n_obs × n_vars = 19942 × 81589
    obs: 'cell_id', 'age', 'batch', 'cell_ontology_class', 'method', 'mouse.id', 'sex', 'tissue', 'old_cell_id_index', 'cell_clean', 'cell_id_index', 'subtissue_clean', 'cell_type_grouped'
    var: 'junction_id', 'event_id', 'splice_motif', 'label_5_prime', 'label_3_prime', 'annotation_status', 'gene_name', 'gene_id', 'num_junctions', 'position_off_5_prime', 'position_off_3_prime', 'CountJuncs', 'old_junction_id_index', 'junction_id_index'
    uns: 'age_colors', 'cell_type_grouped_colors', 'diffmap_evals', 'neighbors', 'pca_explained_variance_ratio', 'tissue_colors', 'tsne', 'umap'
    obsm: 'X_diffmap', 'X_pca', 'X_tsne', 'X_umap', 'phi_init_100_waypoints', 'phi_init_30_waypoints'
    varm: 'psi_init_100_waypoints', 'psi_init_30_waypoints'
    layers: 'cell_by_cluster_matrix', 'cell_by_junction_matrix', 'junc_ratio'
    obsp: 'connectivities', 'distances'

In [3]:
atse_anndata.var

,junction_id,event_id,splice_motif,label_5_prime,label_3_prime,annotation_status,gene_name,gene_id,num_junctions,position_off_5_prime,position_off_3_prime,CountJuncs,old_junction_id_index,junction_id_index
0,chr10_100087456_100088152_+,ATSE_23617,GT-AG,annotated on 5',annotated on 3',both,Kitl,ENSMUSG00000019966.18,3,0.0,1.0,275956,17,0
1,chr10_100087456_100089195_+,ATSE_23617,GT-AG,annotated on 5',annotated on 3',both,Kitl,ENSMUSG00000019966.18,3,0.0,1.0,668,18,1
2,chr10_100088220_100089195_+,ATSE_23617,GT-AG,annotated on 5',annotated on 3',both,Kitl,ENSMUSG00000019966.18,3,0.0,1.0,273540,19,2
3,chr10_100089271_100090054_+,ATSE_23616,GT-AG,annotated on 5',unannotated on 3',five_prime,Kitl,ENSMUSG00000019966.18,2,0.0,NaN,599,20,3
4,chr10_100089271_100096017_+,ATSE_23616,GT-AG,annotated on 5',annotated on 3',both,Kitl,ENSMUSG00000019966.18,2,0.0,1.0,235610,21,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81584,chrY_90840659_90841656_+,ATSE_42712,GT-AG,unannotated on 5',annotated on 3',three_prime,Gm21742,ENSMUSG00000099871.1,6,NaN,1.0,8319,270415,81584
81585,chrY_910586_910656_+,ATSE_47491,GT-AG,annotated on 5',annotated on 3',both,Kdm5d,ENSMUSG00000056673.14,2,0.0,1.0,463,270416,81585
81586,chrY_910586_910685_+,ATSE_47491,GT-AG,annotated on 5',annotated on 3',both,Kdm5d,ENSMUSG00000056673.14,2,0.0,1.0,4867,270417,81586
81587,chrY_927439_927602_+,ATSE_47490,GT-AG,annotated on 5',annotated on 3',both,Kdm5d,ENSMUSG00000056673.14,2,0.0,1.0,1048,270418,81587


In [4]:
atse_anndata.obs

,cell_id,age,batch,cell_ontology_class,method,mouse.id,sex,tissue,old_cell_id_index,cell_clean,cell_id_index,subtissue_clean,cell_type_grouped
cell_id_for_index,,,,,,,,,,,,,
A11_MAA000926,A11-MAA000926-3_9_M-1-1,3m,1,endothelial cell,facs,3_9_M,male,Brain_Non-Myeloid,0,A11_MAA000926,0,Cortex,ENDOTHELIAL CELL
A11_B000176,A11_B000176_B008056_S11,18m,1,endothelial cell,facs,18_53_M,male,Brain_Non-Myeloid,1,A11_B000176,1,Cortex,ENDOTHELIAL CELL
A12_D045318,A12_D045318_B008591_S12,18m,1,microglial cell,facs,18_45_M,male,Brain_Myeloid,2,A12_D045318,2,Striatum,MICROGLIA
A13_B001060,A13_B001060_B009250_S217,18m,1,microglial cell,facs,18_47_F,female,Brain_Myeloid,3,A13_B001060,3,Cerebellum,MICROGLIA
A14_MAA000944,A14-MAA000944-3_9_M-1-1,3m,1,brain pericyte,facs,3_9_M,male,Brain_Non-Myeloid,4,A14_MAA000944,4,Cortex,PERICYTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
P14_B001358,P14_B001358_B008290_S374,18m,1,microglial cell,facs,18_47_F,female,Brain_Myeloid,19937,P14_B001358,19937,Striatum,MICROGLIA
P20_MAA000590,P20-MAA000590-3_9_M-1-1,3m,1,microglial cell,facs,3_9_M,male,Brain_Myeloid,19938,P20_MAA000590,19938,Striatum,MICROGLIA
P20_MAA001894,P20-MAA001894-3_39_F-1-1,3m,1,astrocyte,facs,3_39_F,female,Brain_Non-Myeloid,19939,P20_MAA001894,19939,Cortex,GLIAL CELL


Reading in the gene expression annData. 

In [7]:
ge_anndata = ad.read_h5ad('/gpfs/commons/groups/knowles_lab/Karin/TMS_MODELING/DATA_FILES/BRAIN_ONLY/02072025/TMS_Anndata_GeneExpression_2025-02-07.h5ad')
ge_anndata

AnnData object with n_obs × n_vars = 19942 × 22141
    obs: 'FACS.selection', 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'sex', 'subtissue', 'tissue', 'n_genes', 'n_counts', 'cell_clean', 'cell_id', 'batch', 'old_cell_id_index', 'cell_id_index', 'subtissue_clean', 'cell_type_grouped'
    var: 'n_cells', 'mouse_gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
    layers: 'raw_counts'

In [8]:
ge_anndata.var

,n_cells,mouse_gene_name,highly_variable,means,dispersions,dispersions_norm
index,,,,,,
0610005C13Rik,142,0610005C13Rik,False,0.003122,0.675754,-0.458931
0610007C21Rik,11788,0610007C21Rik,False,1.325835,2.033707,-0.718628
0610007L01Rik,3883,0610007L01Rik,False,0.347774,1.877960,0.314969
0610007N19Rik,695,0610007N19Rik,True,0.052839,3.353300,1.264694
0610007P08Rik,1330,0610007P08Rik,False,0.085841,2.024538,0.409326
...,...,...,...,...,...,...
Zyg11a,19,Zyg11a,False,0.000594,2.013033,0.401920
Zyg11b,3649,Zyg11b,False,0.266270,2.197219,0.520487
Zyx,5704,Zyx,False,0.520497,2.303264,0.058897


In [9]:
ge_anndata.obs

,FACS.selection,age,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,sex,subtissue,tissue,n_genes,n_counts,cell_clean,cell_id,batch,old_cell_id_index,cell_id_index,subtissue_clean,cell_type_grouped
cell_id_for_index,,,,,,,,,,,,,,,,,,,,
A11_MAA000926,nan,3m,A11.MAA000926.3_9_M.1.1,endothelial cell,CL:0000115,Endothelial cell,facs,3_9_M,male,Cortex,Brain_Non-Myeloid,4484,725558.0,A11_MAA000926,A11-MAA000926-3_9_M-1-1,1,0,0,Cortex,ENDOTHELIAL CELL
A11_B000176,Neurons,18m,A11_B000176,endothelial cell,nan,Endothelial cell,facs,18_53_M,male,Cortex,Brain_Non-Myeloid,2177,1173272.0,A11_B000176,A11_B000176_B008056_S11,1,1,1,Cortex,ENDOTHELIAL CELL
A12_D045318,Microglia,18m,A12_D045318,microglial cell,nan,nan,facs,18_45_M,male,Striatum,Brain_Myeloid,1806,821475.0,A12_D045318,A12_D045318_B008591_S12,1,2,2,Striatum,MICROGLIA
A13_B001060,Microglia,18m,A13_B001060,microglial cell,nan,nan,facs,18_47_F,female,Cerebellum,Brain_Myeloid,1660,391823.0,A13_B001060,A13_B001060_B009250_S217,1,3,3,Cerebellum,MICROGLIA
A14_MAA000944,nan,3m,A14.MAA000944.3_9_M.1.1,brain pericyte,CL:0000128,Pericyte,facs,3_9_M,male,Cortex,Brain_Non-Myeloid,1936,1099688.0,A14_MAA000944,A14-MAA000944-3_9_M-1-1,1,4,4,Cortex,PERICYTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P14_B001358,Microglia,18m,P14_B001358,microglial cell,nan,nan,facs,18_47_F,female,Striatum,Brain_Myeloid,1563,265486.0,P14_B001358,P14_B001358_B008290_S374,1,19937,19937,Striatum,MICROGLIA
P20_MAA000590,nan,3m,P20.MAA000590.3_9_M.1.1,microglial cell,CL:0000129,nan,facs,3_9_M,male,Striatum,Brain_Myeloid,2042,1261966.0,P20_MAA000590,P20-MAA000590-3_9_M-1-1,1,19938,19938,Striatum,MICROGLIA
P20_MAA001894,nan,3m,P20.MAA001894.3_39_F.1.1,astrocyte,CL:0000127,nan,facs,3_39_F,female,Cortex,Brain_Non-Myeloid,4184,438825.0,P20_MAA001894,P20-MAA001894-3_39_F-1-1,1,19939,19939,Cortex,GLIAL CELL


1. Preparing var data. As per the AnnData used in MultiVI, we have ID, modality, and n_cells with the index corresponding
to the exact data point (gene or junction id)

In [24]:
gene_expr_var = pd.DataFrame({
    'ID': ge_anndata.var['mouse_gene_name'],
    'modality': 'Gene_Expression'
})
gene_expr_var.index = ge_anndata.var.index

In [25]:
splicing_var = pd.DataFrame({
    'ID': atse_anndata.var['junction_id'],
    'modality': 'Splicing'
})
splicing_var.index = atse_anndata.var.index  

In [26]:
combined_var = pd.concat([gene_expr_var, splicing_var])

2. Preparing obs data. 

In [27]:
combined_obs = pd.DataFrame({
    'batch_id': atse_anndata.obs['batch'],
    'age': atse_anndata.obs['age'],
    'cell_ontology_class': atse_anndata.obs['cell_ontology_class'],
    'cell_type_grouped': atse_anndata.obs['cell_type_grouped'],
    'mouse.id': atse_anndata.obs['mouse.id'],
    'sex': atse_anndata.obs['sex'],
    'modality': 'paired'
})
combined_obs.index = atse_anndata.obs.index

3. Getting Count Data

In [28]:
splicing_counts = atse_anndata.layers['junc_ratio']
gene_expr_counts = ge_anndata.layers['raw_counts']
combined_counts = sp.hstack([gene_expr_counts, splicing_counts])

Creating our combined annData!

In [29]:
combined_adata = ad.AnnData(
    X=combined_counts,
    obs=combined_obs,
    var=combined_var
)

In [30]:
print(combined_adata)

AnnData object with n_obs × n_vars = 19942 × 103730
    obs: 'batch_id', 'age', 'cell_ontology_class', 'cell_type_grouped', 'mouse.id', 'sex', 'modality'
    var: 'ID', 'modality'


Adding padded layers for our ATSE(cluster) counts and junction counts. 

In [31]:
padding_amount = ge_anndata.layers['raw_counts'].shape[1]
print(padding_amount)

22141


In [32]:

from scipy.sparse import csr_matrix
from scipy.sparse import hstack, vstack, csr_matrix

cell_by_junction_matrix = atse_anndata.layers['cell_by_junction_matrix']
cell_by_cluster_matrix = atse_anndata.layers['cell_by_cluster_matrix']

num_rows, num_cols = cell_by_junction_matrix.shape
padding_cols = padding_amount

print(padding_cols)

padding_matrix = csr_matrix((num_rows, padding_cols))

cell_by_junction_matrix_padded = hstack([padding_matrix, cell_by_junction_matrix], format='csr')
cell_by_cluster_matrix_padded = hstack([padding_matrix, cell_by_cluster_matrix], format='csr')

combined_adata.layers['cell_by_junction_matrix'] = cell_by_junction_matrix_padded
combined_adata.layers['cell_by_cluster_matrix'] = cell_by_cluster_matrix_padded

22141


In [33]:
combined_adata.var

,ID,modality
0610005C13Rik,0610005C13Rik,Gene_Expression
0610007C21Rik,0610007C21Rik,Gene_Expression
0610007L01Rik,0610007L01Rik,Gene_Expression
0610007N19Rik,0610007N19Rik,Gene_Expression
0610007P08Rik,0610007P08Rik,Gene_Expression
...,...,...
81584,chrY_90840659_90841656_+,Splicing
81585,chrY_910586_910656_+,Splicing
81586,chrY_910586_910685_+,Splicing
81587,chrY_927439_927602_+,Splicing


In [34]:
combined_adata.obs

,batch_id,age,cell_ontology_class,cell_type_grouped,mouse.id,sex,modality
cell_id_for_index,,,,,,,
A11_MAA000926,1,3m,endothelial cell,ENDOTHELIAL CELL,3_9_M,male,paired
A11_B000176,1,18m,endothelial cell,ENDOTHELIAL CELL,18_53_M,male,paired
A12_D045318,1,18m,microglial cell,MICROGLIA,18_45_M,male,paired
A13_B001060,1,18m,microglial cell,MICROGLIA,18_47_F,female,paired
A14_MAA000944,1,3m,brain pericyte,PERICYTE,3_9_M,male,paired
...,...,...,...,...,...,...,...
P14_B001358,1,18m,microglial cell,MICROGLIA,18_47_F,female,paired
P20_MAA000590,1,3m,microglial cell,MICROGLIA,3_9_M,male,paired
P20_MAA001894,1,3m,astrocyte,GLIAL CELL,3_39_F,female,paired


In [35]:
combined_adata

AnnData object with n_obs × n_vars = 19942 × 103730
    obs: 'batch_id', 'age', 'cell_ontology_class', 'cell_type_grouped', 'mouse.id', 'sex', 'modality'
    var: 'ID', 'modality'
    layers: 'cell_by_junction_matrix', 'cell_by_cluster_matrix'

Writing out our completely processed combined_annData object. 

In [36]:
combined_adata.write("/gpfs/commons/groups/knowles_lab/Karin/TMS_MODELING/DATA_FILES/BRAIN_ONLY/02072025/TMS_BRAINONLY_Combined_GE_ATSE.h5ad")

In [37]:
combined_adata = ad.read_h5ad('/gpfs/commons/groups/knowles_lab/Karin/TMS_MODELING/DATA_FILES/BRAIN_ONLY/02072025/TMS_BRAINONLY_Combined_GE_ATSE.h5ad')

In [38]:
combined_adata

AnnData object with n_obs × n_vars = 19942 × 103730
    obs: 'batch_id', 'age', 'cell_ontology_class', 'cell_type_grouped', 'mouse.id', 'sex', 'modality'
    var: 'ID', 'modality'
    layers: 'cell_by_cluster_matrix', 'cell_by_junction_matrix'